# Cleaning Enron Dataset
In this notebook, we will explore different aspects of the enron data set to infer different information. We will be analyzing and simplifying every column to retrieve different information.

Let's begin by importing some of the required modules.

In [1]:
import email, re
import numpy as np
import pandas as pd
import matplotlib.pyplot as plt
import seaborn as sns
sns.set()
%matplotlib inline

Let's import the processed pickle from previous notebook and check it.

In [35]:
data = pd.read_pickle('data/emails-processed.pkl')
data.head()

,Date,From,To,Subject,X-From,X-To,X-cc,X-bcc,X-Folder,X-Origin,X-FileName,content,user
Message-ID,,,,,,,,,,,,,
<18782981.1075855378110.JavaMail.evans@thyme>,2001-05-14 23:39:00,(phillip.allen@enron.com),(tim.belden@enron.com),,Phillip K Allen,Tim Belden <Tim Belden/Enron@EnronXGate>,,,"\Phillip_Allen_Jan2002_1\Allen, Phillip K.\'Se...",Allen-P,pallen (Non-Privileged).pst,Here is our forecast\n\n,allen-p
<15464986.1075855378456.JavaMail.evans@thyme>,2001-05-04 20:51:00,(phillip.allen@enron.com),(john.lavorato@enron.com),Re:,Phillip K Allen,John J Lavorato <John J Lavorato/ENRON@enronXg...,,,"\Phillip_Allen_Jan2002_1\Allen, Phillip K.\'Se...",Allen-P,pallen (Non-Privileged).pst,Traveling to have a business meeting takes the...,allen-p
<24216240.1075855687451.JavaMail.evans@thyme>,2000-10-18 10:00:00,(phillip.allen@enron.com),(leah.arsdall@enron.com),Re: test,Phillip K Allen,Leah Van Arsdall,,,\Phillip_Allen_Dec2000\Notes Folders\'sent mail,Allen-P,pallen.nsf,test successful. way to go!!!,allen-p
<13505866.1075863688222.JavaMail.evans@thyme>,2000-10-23 13:13:00,(phillip.allen@enron.com),(randall.gay@enron.com),,Phillip K Allen,Randall L Gay,,,\Phillip_Allen_Dec2000\Notes Folders\'sent mail,Allen-P,pallen.nsf,"Randy,\n\n Can you send me a schedule of the s...",allen-p
<30922949.1075863688243.JavaMail.evans@thyme>,2000-08-31 12:07:00,(phillip.allen@enron.com),(greg.piper@enron.com),Re: Hello,Phillip K Allen,Greg Piper,,,\Phillip_Allen_Dec2000\Notes Folders\'sent mail,Allen-P,pallen.nsf,Let's shoot for Tuesday at 11:45.,allen-p


The describe() method will provide us with a simple summary of the data set. We can see the count of the non-empty values present in each columns.

In [3]:
data.describe()

,Date,From,To,Subject,X-From,X-To,X-cc,X-bcc,X-Folder,X-Origin,X-FileName,content,user
count,517401,517401,495554,517401,517372,517372,517372,517372,517372,517372,517372,517401,517401
unique,224122,20328,54748,159290,27980,73552,33701,132,5335,259,429,249025,150
top,2001-06-27 23:02:00,(kay.mann@enron.com),(pete.davis@enron.com),,Kay Mann,,,,\Kay_Mann_June2001_1\Notes Folders\All documents,Kaminski-V,vkamins.nsf,"As you know, Enron Net Works (ENW) and Enron G...",kaminski-v
freq,1118,16735,9155,19187,16324,9124,388486,517197,6639,26995,24029,112,28465
first,1980-01-01 00:00:00,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
last,2044-01-04 22:48:58,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN


From above table, we can see that there are total `517401` number of records in total. Many fields seem to be affected with null values, particularly the `X-bcc`, `X-cc` and `X-To` columns. Many emails have null values in the `To` fields, which cannot be used in our analysis since we cannot deduct who the mail was sent to.

In [4]:
data[data['To'].isnull()][:1]

,Date,From,To,Subject,X-From,X-To,X-cc,X-bcc,X-Folder,X-Origin,X-FileName,content,user
Message-ID,,,,,,,,,,,,,
<15201149.1075855691021.JavaMail.evans@thyme>,2000-05-01 10:56:00,(phillip.allen@enron.com),None,Re: DSL- Installs,Phillip K Allen,Circuit Provisioning@ENRON,,,\Phillip_Allen_Dec2000\Notes Folders\'sent mail,Allen-P,pallen.nsf,No one will be home on 5/11/00 to meet DSL ins...,allen-p


The above example shows that the mail was sent by phillip.allen@enron.com, but the receiver is not mentioned. Such emails cannot be used in network analysis of the users since we cannot define a node using the null values.

Let's group the records by the Date the email was sent on. The following table shows the number of emails sent on each date.

In [5]:
emails_by_date = data.groupby('Date').count()
emails_by_date[:5]

,From,To,Subject,X-From,X-To,X-cc,X-bcc,X-Folder,X-Origin,X-FileName,content,user
Date,,,,,,,,,,,,
1980-01-01 00:00:00,522,316,522,522,522,522,522,522,522,522,522,522
1986-04-26 15:22:07,1,1,1,1,1,1,1,1,1,1,1,1
1986-05-01 14:37:34,1,1,1,1,1,1,1,1,1,1,1,1
1997-01-01 17:22:03,1,1,1,1,1,1,1,1,1,1,1,1
1997-03-03 11:00:00,3,0,3,3,3,3,3,3,3,3,3,3


It seems that `522` emails were sent on `1980-01-01`, but the Enron corporation was formed on `1985`. It seems that the Date column is also filled with mis-labeled data.

There's a lot to do before we can use the dataset. Firstly, we'll use Regular expressions to extract the user name, the domain and the top level domains from each email. Following is the domain and email user name regex which will be combined to obtain the regular expression for the email.

In [6]:
domainRegex = r"(?P<full_domain>(?P<domain>(?:(?!-)(?:xn--|_{1,1})?[a-z0-9-]{0,61}[a-z0-9]{1,1}\.)*)(?P<tld>(?:xn--)?[a-z0-9\-]{1,61}|[a-z0-9-]{1,30}\.[a-z]{2,}))"
emailUsernameRegex = r"(?P<username>[a-zA-Z0-9.!#$%&*+/=?^_`|}~-]+)"
emailRegex = emailUsernameRegex + "@" + domainRegex + r""

Then, we will need to compile the regular expressions to use it. We will use the python `re` module for it.

In [7]:
email_regex = re.compile(emailRegex)

Let's create a python dictionary to temporarily store values extracted from the emails using the regular expression. The keys of the dictionary will be the extracted item from the email.

In [56]:
emails_data = {
    "from_username": [],
    "from_full_domain": [],
    "from_domain": [],
    "from_tld": [],
    "to_username": [],
    "to_full_domain": [],
    "to_domain": [],
    "to_tld": []
}

First, lets apply the regular expression on the From field of the emails. Any null values or anything not an email will be replaced with empty strings.

In [59]:
for emails in data.From:
    for email in emails:
        try:
            items = list(email_regex.search(email).groups())
        except AttributeError:
            items = ['', '', '', '']
        emails_data['from_username'].append(items[0])
        emails_data['from_full_domain'].append(items[1])
        emails_data['from_domain'].append(items[2][:-1])
        emails_data['from_tld'].append(items[3])

Let's repeat the process for the To field. We'll need to some extra work to manage the multiple emails in the To fields.

In [57]:
for emails in data.To:
    try:
        if len(emails)>1:
            username_list, fdomain_list, domain_list, tld_list = [], [], [], []
            for email in emails:
                items = list(email_regex.search(str(email)).groups())
                username_list.append(items[0])
                fdomain_list.append(items[1])
                domain_list.append(items[2][:-1])
                tld_list.append(items[3])
            items = [username_list, fdomain_list, domain_list, tld_list]
        else:
            for email in emails:
                items = list(email_regex.search(str(email)).groups())
    except (TypeError, AttributeError):
        items = ['', '', '', '']
        
    emails_data['to_username'].append(items[0])
    emails_data['to_full_domain'].append(items[1])
    emails_data['to_domain'].append(items[2][:-1])
    emails_data['to_tld'].append(items[3])

After extracting the respective values, we can then create a pandas DataFrame using the dictionary using the `pd.DataFrame.from_dict()` method. The column names will be the key names from the dictionary. Then, we'll set the index same as that of our `data` dataframe to help in concatenation.

In [61]:
emails_df = pd.DataFrame.from_dict(emails_data, orient='columns')
emails_df.index = data.index

Then, we will concatenate the `data` and `emails_df` dataframes to create our new `data` dataframe.

In [62]:
data = pd.concat([data, emails_df], axis=1)

Let's check some records.

In [68]:
data.head()

,Date,From,To,Subject,X-From,X-To,X-cc,X-bcc,X-Folder,X-Origin,...,content,user,from_domain,from_full_domain,from_tld,from_username,to_domain,to_full_domain,to_tld,to_username
Message-ID,,,,,,,,,,,,,,,,,,,,,
<18782981.1075855378110.JavaMail.evans@thyme>,2001-05-14 23:39:00,(phillip.allen@enron.com),(tim.belden@enron.com),,Phillip K Allen,Tim Belden <Tim Belden/Enron@EnronXGate>,,,"\Phillip_Allen_Jan2002_1\Allen, Phillip K.\'Se...",Allen-P,...,Here is our forecast\n\n,allen-p,enron,enron.com,com,phillip.allen,enron,enron.com,com,tim.belden
<15464986.1075855378456.JavaMail.evans@thyme>,2001-05-04 20:51:00,(phillip.allen@enron.com),(john.lavorato@enron.com),Re:,Phillip K Allen,John J Lavorato <John J Lavorato/ENRON@enronXg...,,,"\Phillip_Allen_Jan2002_1\Allen, Phillip K.\'Se...",Allen-P,...,Traveling to have a business meeting takes the...,allen-p,enron,enron.com,com,phillip.allen,enron,enron.com,com,john.lavorato
<24216240.1075855687451.JavaMail.evans@thyme>,2000-10-18 10:00:00,(phillip.allen@enron.com),(leah.arsdall@enron.com),Re: test,Phillip K Allen,Leah Van Arsdall,,,\Phillip_Allen_Dec2000\Notes Folders\'sent mail,Allen-P,...,test successful. way to go!!!,allen-p,enron,enron.com,com,phillip.allen,enron,enron.com,com,leah.arsdall
<13505866.1075863688222.JavaMail.evans@thyme>,2000-10-23 13:13:00,(phillip.allen@enron.com),(randall.gay@enron.com),,Phillip K Allen,Randall L Gay,,,\Phillip_Allen_Dec2000\Notes Folders\'sent mail,Allen-P,...,"Randy,\n\n Can you send me a schedule of the s...",allen-p,enron,enron.com,com,phillip.allen,enron,enron.com,com,randall.gay
<30922949.1075863688243.JavaMail.evans@thyme>,2000-08-31 12:07:00,(phillip.allen@enron.com),(greg.piper@enron.com),Re: Hello,Phillip K Allen,Greg Piper,,,\Phillip_Allen_Dec2000\Notes Folders\'sent mail,Allen-P,...,Let's shoot for Tuesday at 11:45.,allen-p,enron,enron.com,com,phillip.allen,enron,enron.com,com,greg.piper
